In [1]:
# STL
import os
from typing import Optional, Dict, Any
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
# 3rd Party
import torch
from torch.utils.data import DataLoader
torch.manual_seed(0)
from transformers import BertModel, AutoModel, BertTokenizerFast, AutoTokenizer, PreTrainedTokenizerFast
from tokenizers.pre_tokenizers import BertPreTokenizer, PreTokenizer
# Local
from gatbert.constants import DEFAULT_MODEL, Stance, NodeType, NODE_PAD_ID
from gatbert.data import *
from gatbert.rgat_layer import RGATLayer

/home/ethanlmines/blue_dir/conda_envs/gatbert/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class GATBert(torch.nn.Module):
    def __init__(self,
                 pretrained_model: str,
                 n_relations: int,
                 n_kb_nodes: int,
                 n_classes: int,
                 n_bases: int = 20,
                 ):
        """
        Args:
            n_kb_nodes: Number of unique kb nodes (including the padding node)
            n_relation: Number of unique relations (including the padding relation)

        Relation 0 represents token-to-token
        """
        super().__init__()
        self.__n_relations = n_relations
        self.__bert_model = BertModel.from_pretrained(pretrained_model)
        self.__feature_size: int = self.__bert_model.config.hidden_size
        self.__n_classes = n_classes

        self.__pad_token_id: int = self.__bert_model.config.pad_token_id

        self.__kb_embeddings = torch.nn.Embedding(
            num_embeddings=n_kb_nodes,
            embedding_dim=self.__feature_size,
            padding_idx=NODE_PAD_ID
        )

        self.__rgat = RGATLayer(
            in_features=self.__feature_size,
            attention_units=self.__feature_size,
            out_features=self.__feature_size,
            n_heads=6,
            n_relations=self.__n_relations,
            n_bases=n_bases,
            attention_mode='wirgat'
        )

        self.__projection = torch.nn.Linear(self.__feature_size, self.__n_classes)

    def forward(self,
                 kb_ids: torch.Tensor,
                 edge_indices: torch.Tensor,
                 input_ids: torch.Tensor,
                 attention_mask: torch.Tensor,
                 **bert_kwargs):
        """

        Args:
            kb_ids: (batch, max_external_nodes) array with the IDs of knowledge base nodes
            edge_indices: (?, 6) index array with latter 6 components being (batch, head, tail, relation, head_node_type, tail_node_type)
                batch indexes into the batch dim of other arguments
                head indexes into the seq dim of input_ids, or the node dim of kb_ids
                tail indexes into the seq dim of input_ids, or the node dim of kb_ids
                head_node_type indicates whether head indexes into input_ids or kb_ids
                tail_node_type indicates whether tail indexes into input_ids or kb_ids
            input_ids: (batch, max_sequence_length) array with token IDs
                Indexes into our token embeddings
            attention_mask: attention mask array for input_ids
            bert_kwargs: kwargs to pass on to underlying bert model
        Returns:
            (batch, 3) array of stance class log probabilities
        """

        bert_encodings: torch.Tensor = self.__bert_model(input_ids=input_ids,
                                                             attention_mask=attention_mask,
                                                             **bert_kwargs).last_hidden_state
        node_embeddings = self.__kb_embeddings(kb_ids)

        not_token_padding = input_ids != self.__pad_token_id
        # Number of tokens in each sequence of the batch.
        n_tokens = torch.sum(not_token_padding, dim=-1)
        not_node_padding = kb_ids != NODE_PAD_ID
        # Number of external nodes in each sequence of the batch
        n_kb_nodes = torch.sum(not_node_padding, dim=-1)
        max_graph_size = torch.max(n_tokens + n_kb_nodes)

        # Pack the features from tokens and external KB nodes together into one feature map
        # All features are zero by default
        node_features = torch.zeros([input_ids.shape[0], max_graph_size, self.__feature_size])
        # The first nodes' features should be from tokens
        batch_token_indices, token_indices = torch.where(not_token_padding)
        node_features[batch_token_indices, token_indices] = bert_encodings[batch_token_indices, token_indices]
        # The immediately subsequent nodes should have KB features
        batch_node_indices, node_indices = torch.where(not_node_padding)
        translated_node_indices = n_tokens[batch_node_indices] + node_indices
        node_features[batch_node_indices, translated_node_indices] += node_embeddings[batch_node_indices, node_indices]
        # Everything else following is zeros, since we used torch.zeros() to construct node_embeddings

        # The GAT layer doesn't take node types--it expects a unique ID for each node
        # Thus, we take the number of token nodes for a sequence, and use that as an
        # offset for the IDs of KB nodes
        kb_head_indices = torch.where(edge_indices[4] == NodeType.KB.value)[0]
        head_offsets = n_tokens[edge_indices[0, kb_head_indices]]
        edge_indices[1, kb_head_indices] += head_offsets

        kb_tail_indices = torch.where(edge_indices[5] == NodeType.KB.value)[0]
        tail_offsets = n_tokens[edge_indices[0, kb_tail_indices]]
        edge_indices[2, kb_tail_indices] += tail_offsets

        edges_tensor = torch.sparse_coo_tensor(
            indices=edge_indices[:4],
            values=torch.ones(edge_indices.shape[1]),
            size=(input_ids.shape[0], max_graph_size, max_graph_size, self.__n_relations),
            is_coalesced=True
        )

        last_node_states = self.__rgat(node_features, edges_tensor)
        # The first node will correspond to the first token of the sequence, which is [CLS] for Bert models
        cls_node_features = last_node_states[:, 0]
        logits = self.__projection(cls_node_features)
        return logits

In [3]:
pretrained_model_name = 'bert-base-cased'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name, use_fast=True)

In [ ]:
encoder = make_encoder(tokenizer, BertPreTokenizer())
fake_samples = [
    Sample(
        context="We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness.",
        target="Independence from Britain",
        stance=Stance.FAVOR
    ),
    Sample(
        context="Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal.",
        target="Social Security",
        stance=Stance.NONE
    )
]
ds = MapDataset([encoder(s) for s in fake_samples])
loader = DataLoader(ds, batch_size=2, shuffle=False, collate_fn=make_collate_fn(tokenizer))

In [6]:
gat_model = GATBert(
    pretrained_model=pretrained_model_name,
    n_relations=len(DummyRelationType),
    n_kb_nodes=1001,
    n_classes=len(Stance)
)

In [7]:
for batch in loader:
    stance = batch.pop('stance')
    output = gat_model(**batch)

In [8]:
output.shape

torch.Size([2, 3])